###This notebook demonstrated how you can use the Adversarial Simulator to test your LLM application.
For more information: https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/simulator-interaction-data


#!pip install promptflow-evals

In [ ]:
from promptflow.evals.synthetic import AdversarialSimulator
from typing import List, Dict, Any
from pizza import orderPizza
import os

In [ ]:
from azure.identity import DefaultAzureCredential

azure_ai_project = {
    "subscription_id": os.environ.get('SUBSCRIPTION_ID'),
    "resource_group_name": os.environ.get('RESOURCE_GROUP_NAME'),
    "project_name": os.environ.get('PROJECT_NAME'),
    "credential": DefaultAzureCredential(),
}

In [ ]:
async def callback(
    messages: List[Dict],
    stream: bool = False,
    session_state: Any = None,
) -> dict:
     messages_list = messages["messages"]
    # get last message
    latest_message = messages_list[-1]
    query = latest_message["content"]

    # Add file contents for summarization or re-write
    if 'file_content' in messages["template_parameters"]:
        query += messages["template_parameters"]['file_content']
    
    # Call your own endpoint and pass your query as input. Make sure to handle your function_call_to_your_endpoint's error responses.
    response = await orderPizza(query) 
    
    # Format responses in OpenAI message protocol
    formatted_response = {
        "content": response,
        "role": "assistant",
        "context": {},
    }

    messages["messages"].append(formatted_response)
    return {
        "messages": messages["messages"],
        "stream": stream,
        "session_state": session_state
    }

In [ ]:
from promptflow.evals.synthetic import AdversarialScenario

scenario = AdversarialScenario.ADVERSARIAL_CONVERSATION
simulator = AdversarialSimulator(azure_ai_project=azure_ai_project)

outputs = await simulator(
        scenario=scenario, # required adversarial scenario to simulate
        target=callback, # callback function to simulate against
        max_conversation_turns=3, #optional, applicable only to conversation scenario
        max_simulation_results=1, #optional
        jailbreak=False #optional
    )

# By default simulator outputs json, use the following helper function to convert to QA pairs in jsonl format
print(outputs.to_eval_qa_json_lines())